In [2]:
import pandas as pd
import numpy as np

# Counting the total number of flights between cities

In [3]:
flights_count = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()  # 用size 统计各组行数
flights_count.head()

ORG_AIR  DEST_AIR
ATL      ABE         31
         ABQ         16
         ABY         19
         ACY          6
         AEX         40
dtype: int64

In [4]:
flights_count.loc[[('ATL', 'IAH'), ('IAH', 'ATL')]]  # 传入元组

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

In [95]:
flights[['ORG_AIR', 'DEST_AIR']].head()

,ORG_AIR,DEST_AIR
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DFW,DCA
4,LAX,MCI


In [6]:
#注意，此处不同于groupby后的apply。
flights_sorted = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)#不论出发地、目的地，左右排序。
sort(flights.loc[0, ['ORG_AIR', 'DEST_AIR']])   # 之所以用apply，是因为基于每一行排序，从上到下遍历每行，而不是基于一列。
flights_sorted.head()

#.sort_values做不到，所以apply(sort,axis=1)
#flights[['ORG_AIR', 'DEST_AIR']].sort_values(by=['ORG_AIR', 'DEST_AIR'],axis=1) 

,ORG_AIR,DEST_AIR
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI


但是，上面的操作代价很高，这里仅有6w行，若数量更多，方法 **.apply(,axis=1)** 是Pandas中性能最差的操作之一。
Pandas 内部循环遍历每一行，没有采用任何numpy的加速措施。尽可能 avoid using **apply** with **axis=1**。

In [7]:
flights_count_ord = flights_sorted.rename(columns={'ORG_AIR':'AIR1', 'DEST_AIR':'AIR2'}) \ # 重命名列名
                                  .groupby(['AIR1', 'AIR2']) \
                                  .size()
flights_count_ord.head(10)

AIR1  AIR2
ABE   ATL     31
      ORD     24
ABI   DFW     74
ABQ   ATL     16
      DEN     46
      DFW     54
      IAH     26
      LAS     35
      LAX     55
      MSP      9
dtype: int64

In [8]:
flights_count_ord.loc[('ATL', 'IAH')]  # 查看 Altanta 与 Houston 往来的航班数

269

In [9]:
flights_count_ord.loc[('IAH', 'ATL')]

IndexingError: Too many indexers

## There's more

用numpy 优化上述排序操作

In [30]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

In [34]:
flights_sorted2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
flights_sorted2.equals(flights_sorted.rename(columns={'ORG_AIR':'AIR1', 'DEST_AIR':'AIR2'}))

True

In [37]:
%timeit flights_sorted = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)

7.41 s ± 189 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sorted2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])

10.6 ms ± 453 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
